# **Veri Seti Eğitimi Yolov5**

# **Kurulum**

In [ ]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # repostory'i klonluyoruz
%cd yolov5
%pip install -qr requirements.txt # gerekli kütüphaneleri install ediyoruz
%pip install -q roboflow #roboflow u install ediyoruz

import torch
import os
from IPython.display import Image, clear_output  # resimleri görüntülemek için

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})") #hangi ekran kartını kullandığımızı yazdırıyoruz

# **Roboflowu aktif ediyoruz ve roboflowdaki verisetimizi indiriyoruz**

In [ ]:
# Roboflowu kuruyoruz
!pip install roboflow
from roboflow import Roboflow

In [ ]:
# environment'ı kuruyoruzq
os.environ["DATASET_DIRECTORY"] = "/content/dataset" #yüklemek istediğiniz dizini buraya giriyorsunuz. Bu dizin colabtaki dosya yoludur!

In [ ]:
#roboflowdan aldığımız api kodlarını kopyalıyoruz ve direk yapıştırıyoruz. Bu hücreyi çalıştırınca datasetimiz indirilecektir. 
rf = Roboflow(api_key="Your-api-key")
project = rf.workspace("nameproject").project("namedataset")
version = project.version("version")
dataset = version.download("yolov8")


# **Modelin eğitimine başlıyoruz**

In [ ]:
!pip install ultralytics
!pip install super_gradients


In [ ]:
from ultralytics import YOLO

# YOLOv8 modelini yükle veya yolunu belirle
model = YOLO('yolov8n.pt')  # 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt' gibi diğer modelleri de seçebilirsin.

# Modeli eğit
model.train(
    data=f"{dataset.location}/data.yaml",  # Eğitim verisinin yolunu belirt.
    epochs=100,                       # Eğitim için epoch sayısı
    imgsz=640,                       # Resim boyutu
    batch=16,                        # Batch boyutu
    name='yolov8_custom'             # Eğitim sonucu oluşturulacak klasör adı
)



# **Modelin başarı oranını grafikler halinde ekrana bastırıyoruz**

In [ ]:
# tensorboard'ı başlatıyoruz

#"runs" isimli dosyanın içine kaydediyoruz
%load_ext tensorboard
%tensorboard --logdir runs

# **Test verilerimizi deniyoruz**

In [ ]:
#test resimlerimizi modele sokuyoruz ve sonuçlar en sonda belirtilen dosya konumuna kaydediliyor
#her farklı resim veya video eklediğinizde en sondaki exp1 exp2 diye artar. runs/detect/exp
!python detect.py --weights /content/yolov5/best.pt --img 416 --conf 0.2 --source {dataset.location}/valid/images

# **Test verilerinin son halini ekrana bastırıyoruz**

In [ ]:

#modelden çıkmış olan tüm test verilerini ekrana bastırarak sonuçları inceliyoruz.

import glob
from IPython.display import Image, display

i = 0
# lütfen glob.glob un içine doğru dosya konumunu yazınız üst satırda en altta yazan results saved to runs/detects/exp1 veya runs/detects/exp2 veya runs/detects/exp3
for imageName in glob.glob('/content/yolov5/runs/detect/exp6/*.jpg'): #dosya uzantısını resimlerinizin formatına göre değiştirin jpg olmak zorunda değil.
    i += 1

    if i < 12:
      display(Image(filename=imageName))
      print("\n")

# **Modeli kaydediyoruz**

In [ ]:
#Modelin ağırlıklarını daha sonra kullanmak için bilgisayarımıza kaydediyoruz. Colab içindir.
from google.colab import files
files.download('./runs/train/exp/weights/best.pt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>